In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay, easeFunction

objc[61709]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2a2386e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2bc17aea0). One of the two will be used. Which one is undefined.
objc[61709]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2a2386db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2bc17aec8). One of the two will be used. Which one is undefined.
objc[61709]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2a2386e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [6]:
trichromat = Observer.trichromat(wavelengths=wavelengths5, illuminant=d65) 
maxbasis = MaxBasis(trichromat, verbose=True)
viz = PSWrapper(trichromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.MESH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True)

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
maxbasis_tetra = MaxBasis(tetrachromat, verbose=True)

100%|██████████| 58/58 [00:00<00:00, 594.96it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


100%|██████████| 28/28 [00:00<00:00, 62.21it/s]


In [7]:
# helper functions
def SetEnabledFn(list_names, list_types=None):
    if list_types is None:
        return lambda list_true: [viz.ps.get_surface_mesh(item).set_enabled(enabled) for item, enabled in zip(list_names, list_true)]
    else:
        return lambda list_true: [getattr(viz.ps, f'get_{type_name}')(item).set_enabled(enabled) for item, type_name, enabled in zip(list_names, list_types, list_true)]

In [13]:
# general settings
viz.ps.set_ground_plane_mode("none")
dirname = "./outputs/trichromat_presentation_loop"
os.makedirs(dirname, exist_ok=True)
offset = 0

# render all objects
names, mat = viz.lineOfArrows()
mesh_types = ["surface_mesh", "surface_mesh", "curve_network", "surface_mesh"]
set_enabled = SetEnabledFn(names, mesh_types)

In [9]:
set_enabled([False, True, False, False])
viz.ps.screenshot(f"{dirname}/trichromat_chromatic_space.png")
set_enabled([False, True, False, True])
viz.ps.screenshot(f"{dirname}/trichromat_achromatic.png")
set_enabled([True, True, True, False])
viz.ps.screenshot(f"{dirname}/trichromat_confusion_lines.png")

In [23]:
refs, discrete_points, rgbs, lines = maxbasis_tetra.getDiscreteRepresentation()

points = np.array([trichromat.observe(s) for s in refs])
ref_points = (viz.HMatrix@maxbasis.cone_to_maxbasis@points.T).T
ref_points[:, 0] = 0


name = viz.renderPointCloud(ref_points[1:-1], rgbs[1:-1], radius=0.03)
viz.ps.get_point_cloud(name[0]).set_transform(mat)
viz.ps.get_point_cloud(name[0]).set_enabled(True)

In [24]:
set_enabled([False, False, False, False])
viz.ps.screenshot(f"{dirname}/tetrachromat_tetra_lattice.png")

: 

In [20]:
set_enabled([False, True, False, False])
viz.ps.get_point_cloud(name[0]).set_enabled(True)
viz.ps.screenshot(f"{dirname}/tetrachromat_chrom.png")

In [18]:
viz.ps.show()